In [241]:
from sqlalchemy import create_engine, text
from sqlalchemy import inspect
import pandas as pd

In [243]:
engine = create_engine('sqlite:////Users/sarahberkin/Desktop/sqlite_db_pythonsqlite.db')

In [245]:
table_names = inspect(engine).get_table_names()
print(table_names)

['Bookings', 'Facilities', 'Members']


In [247]:
con = engine.connect()
rs = con.execute(text("SELECT name, COUNT(bookid) FROM Bookings LEFT JOIN Facilities USING(facid) WHERE memid IN (1,2) GROUP BY name"))
df = pd.DataFrame(rs.fetchall())

In [249]:
print(df)

              name  COUNT(bookid)
0  Badminton Court            164
1   Massage Room 1             34
2       Pool Table             89
3    Snooker Table             57
4     Squash Court             20
5     Table Tennis             56
6   Tennis Court 1             30
7   Tennis Court 2             21


In [251]:
df = pd.read_sql_query("SELECT name, COUNT(bookid) FROM Bookings LEFT JOIN Facilities USING(facid) WHERE memid IN (1,2) GROUP BY name", engine)

In [253]:
print(df)

              name  COUNT(bookid)
0  Badminton Court            164
1   Massage Room 1             34
2       Pool Table             89
3    Snooker Table             57
4     Squash Court             20
5     Table Tennis             56
6   Tennis Court 1             30
7   Tennis Court 2             21


In [255]:
member_revenue = pd.read_sql_query("SELECT Facilities.name, (COUNT(bookid) * membercost) AS member_cost FROM Bookings LEFT JOIN Facilities USING(facid) WHERE Bookings.memid IN (1,2) GROUP BY name", engine)
print(member_revenue)

              name  member_cost
0  Badminton Court          0.0
1   Massage Room 1        336.6
2       Pool Table          0.0
3    Snooker Table          0.0
4     Squash Court         70.0
5     Table Tennis          0.0
6   Tennis Court 1        150.0
7   Tennis Court 2        105.0


In [257]:
guest_revenue = pd.read_sql_query("SELECT Facilities.name, (COUNT(bookid) * guestcost) AS guest_cost FROM Bookings LEFT JOIN Facilities USING(facid) WHERE Bookings.memid = 0 GROUP BY name", engine)
print(guest_revenue)

              name  guest_cost
0  Badminton Court       604.5
1   Massage Room 1     16640.0
2   Massage Room 2      6720.0
3       Pool Table       265.0
4    Snooker Table       115.0
5     Squash Court      4287.5
6     Table Tennis        90.0
7   Tennis Court 1      2500.0
8   Tennis Court 2      2825.0


In [259]:
all_revenue = pd.merge(member_revenue, guest_revenue, how='inner', on=None)
print(all_revenue)

              name  member_cost  guest_cost
0  Badminton Court          0.0       604.5
1   Massage Room 1        336.6     16640.0
2       Pool Table          0.0       265.0
3    Snooker Table          0.0       115.0
4     Squash Court         70.0      4287.5
5     Table Tennis          0.0        90.0
6   Tennis Court 1        150.0      2500.0
7   Tennis Court 2        105.0      2825.0


In [261]:
all_revenue.loc[:, 'total_revenue'] = all_revenue.sum(numeric_only=True, axis=1)
print(all_revenue)

              name  member_cost  guest_cost  total_revenue
0  Badminton Court          0.0       604.5          604.5
1   Massage Room 1        336.6     16640.0        16976.6
2       Pool Table          0.0       265.0          265.0
3    Snooker Table          0.0       115.0          115.0
4     Squash Court         70.0      4287.5         4357.5
5     Table Tennis          0.0        90.0           90.0
6   Tennis Court 1        150.0      2500.0         2650.0
7   Tennis Court 2        105.0      2825.0         2930.0


In [263]:
final_result_q10 = all_revenue[all_revenue['total_revenue'] < 1000]
print(final_result_q10)

              name  member_cost  guest_cost  total_revenue
0  Badminton Court          0.0       604.5          604.5
2       Pool Table          0.0       265.0          265.0
3    Snooker Table          0.0       115.0          115.0
5     Table Tennis          0.0        90.0           90.0


In [270]:
member_report = pd.read_sql_query("SELECT CONCAT(surname, ', ', firstname) AS member_name, recommendedby AS recommeneded_by FROM Members ORDER BY surname", engine)
print(member_report)

                 member_name recommeneded_by
0            Bader, Florence               9
1                Baker, Anne               9
2             Baker, Timothy              13
3                Boothe, Tim               3
4            Butters, Gerald               1
5               Coplin, Joan              16
6             Crumpet, Erica               2
7                Dare, Nancy               4
8            Farrell, Jemima                
9             Farrell, David                
10              GUEST, GUEST                
11          Genting, Matthew               5
12                Hunt, John              30
13              Jones, David               4
14            Jones, Douglas              11
15          Joplette, Janice               1
16           Mackenzie, Anna               1
17             Owen, Charles               1
18             Pinker, David              13
19        Purview, Millicent               2
20               Rownam, Tim                
21        

In [275]:
usage_by_member = pd.read_sql_query("SELECT Facilities.name AS facility, COUNT(Bookings.bookid) AS usage_by_members FROM Bookings LEFT JOIN Facilities USING(facid) WHERE memid IN (1,2) GROUP BY Facilities.name", engine)
print(usage_by_member)

          facility  usage_by_members
0  Badminton Court               164
1   Massage Room 1                34
2       Pool Table                89
3    Snooker Table                57
4     Squash Court                20
5     Table Tennis                56
6   Tennis Court 1                30
7   Tennis Court 2                21


In [278]:
usage_by_month = pd.read_sql_query("SELECT Facilities.name AS facility, monthlymaintenance AS usage_by_month FROM Bookings LEFT JOIN Facilities USING(facid) WHERE memid IN (1,2) GROUP BY Facilities.name", engine)
print(usage_by_month)

          facility  usage_by_month
0  Badminton Court              50
1   Massage Room 1            3000
2       Pool Table              15
3    Snooker Table              15
4     Squash Court              80
5     Table Tennis              10
6   Tennis Court 1             200
7   Tennis Court 2             200
